# 패키지

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import pickle
import torch.nn.functional as nnf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_path = "/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/train_data_list.pkl"
test_path = "/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/test_data_list.pkl"

# accuracy함수

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    index = 0
    with torch.no_grad():
        for images, labels in loader:
            top2 = []
            X = images.clone().detach()

            x = images.to(device=device)
            y = labels.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)

            index += 1
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100}')

    model.train()
    return num_correct

# train

In [ ]:
# PKL 파일 로드
with open(train_path, 'rb') as f:
    train_data = pickle.load(f)

with open(test_path, 'rb') as f:
    test_data = pickle.load(f)


# 데이터셋 클래스 작성
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # PKL 파일에서 이미지와 레이블을 가져옴
        img, label = self.data[idx]

        # 이미지를 PIL 이미지로 변환
        img = transforms.ToPILImage()(img)

        # 이미지에 정의된 변환을 적용
        if self.transform:
            img = self.transform(img)

        return img, label

rotation_range = (-15, 15)
# 이미지 변환 정의
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(),
     transforms.RandomVerticalFlip(),
     transforms.RandomAffine(degrees=rotation_range),
     transforms.RandomPerspective(distortion_scale=.1, p=.1, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.RandomRotation(degrees=20, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.Normalize(mean=[0.5878, 0.5398, 0.4853],std=[0.1505, 0.1592, 0.1703])
])

sub_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5915, 0.5468, 0.4935],
                             std=[0.1549, 0.1642, 0.1756]),
    ]
)

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=sub_transform)

In [ ]:
# 데이터셋을 학습용과 검증용으로 분할
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 분할된 데이터셋을 DataLoader에 로드할 수 있습니다.
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
in_chnnels = 3
num_classes = 10
learning_rate = 1e-5
batch_size = 8
num_epochs = 100
load_model = False
model = torchvision.models.googlenet(init_weights=True).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, verbose=True)

In [ ]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (images, labels) in enumerate(train_loader):
        # Get data to cuda if possible
        images = images.to(device=device)
        labels = labels.to(device=device)

        # forward
        aux1,aux2, scores = model(images) # googlenet은 output이 3개 입니다.

        loss1 = criterion(scores, labels)
        loss2 = criterion(aux1, labels)
        loss3 = criterion(aux2, labels)
        loss = loss1 + 0.3*(loss2)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f'Cost at epoch {epoch} is {sum(losses) / len(losses)}')

#validation set

In [ ]:
check_accuracy(val_loader,model)

In [ ]:
correct = 0
total = 0
correct_per_label = {label: 0 for label in range(1, 11)}
total_per_label = {label: 0 for label in range(1, 11)}
total_correct = 0
total_samples = 0
model.eval()

# 인퍼런스 모드를 위해 no_grad 해줍니다.
with torch.no_grad():
    # 테스트로더에서 이미지와 정답을 불러옵니다.
    for images,labels in val_loader:
        x = images.to(device)
        y = labels.to(device)

        # 모델에 데이터를 넣고 결과값을 얻습니다.
        output = model.forward(x)
        _,output_index = torch.max(output,1)

        # 전체 개수 += 라벨의 개수
        total += labels.size(0)
        correct += (output_index == y).sum().float()

        for label in range(1, 11):
                total_per_label[label] += (y == label).sum().item()
                correct_per_label[label] += ((y == label) & (output_index == y)).sum().item()
    accuracy_per_label = {label: correct_per_label[label] / total_per_label[label] if total_per_label[label] > 0 else 0.0
                          for label in range(1, 11)}
    overall_accuracy = total_correct / total_samples if total_samples > 0 else 0.0

    for label in range(1, 11):
        print(f"Label {label}: {correct_per_label[label]} / {total_per_label[label]} with accuracy {accuracy_per_label[label] * 100:.2f}%")
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))

# test set

In [ ]:
check_accuracy(test_loader,model)

In [ ]:
correct = 0
total = 0
correct_per_label = {label: 0 for label in range(1, 11)}
total_per_label = {label: 0 for label in range(1, 11)}
total_correct = 0
total_samples = 0
model.eval()

# 인퍼런스 모드를 위해 no_grad 해줍니다.
with torch.no_grad():
    # 테스트로더에서 이미지와 정답을 불러옵니다.
    for images,labels in val_loader:
        x = images.to(device)
        y = labels.to(device)

        # 모델에 데이터를 넣고 결과값을 얻습니다.
        output = model.forward(x)
        _,output_index = torch.max(output,1)

        # 전체 개수 += 라벨의 개수
        total += labels.size(0)
        correct += (output_index == y).sum().float()

        for label in range(1, 11):
                total_per_label[label] += (y == label).sum().item()
                correct_per_label[label] += ((y == label) & (output_index == y)).sum().item()
    accuracy_per_label = {label: correct_per_label[label] / total_per_label[label] if total_per_label[label] > 0 else 0.0
                          for label in range(1, 11)}
    overall_accuracy = total_correct / total_samples if total_samples > 0 else 0.0

    for label in range(1, 11):
        print(f"Label {label}: {correct_per_label[label]} / {total_per_label[label]} with accuracy {accuracy_per_label[label] * 100:.2f}%")
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))

# 3,8,10만 뽑는 코드

In [ ]:
# PKL 파일 로드
with open(train_path, 'rb') as f:
    train_data = pickle.load(f)

with open(test_path, 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
train_data1 = [sample for sample in train_data if sample[1] in [3, 8,10]]
test_data1 = [sample for sample in test_data if sample[1] in [3, 8,10]]

In [ ]:
# 데이터셋 클래스 작성
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # PKL 파일에서 이미지와 레이블을 가져옴
        img, label = self.data[idx]

        # 이미지를 PIL 이미지로 변환
        img = transforms.ToPILImage()(img)

        # 이미지에 정의된 변환을 적용
        if self.transform:
            img = self.transform(img)

        return img, label

rotation_range = (-15, 15)
# 이미지 변환 정의
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(),
     transforms.RandomVerticalFlip(),
     transforms.RandomAffine(degrees=rotation_range),
     transforms.RandomPerspective(distortion_scale=.1, p=.1, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.RandomRotation(degrees=20, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.Normalize(mean=[0.5878, 0.5398, 0.4853],std=[0.1505, 0.1592, 0.1703])
])

sub_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5915, 0.5468, 0.4935],
                             std=[0.1549, 0.1642, 0.1756]),
    ]
)

# 데이터셋 인스턴스 생성
train_dataset1 = CustomDataset(train_data1, transform=transform)
test_dataset1 = CustomDataset(test_data1, transform=sub_transform)

In [ ]:
# 데이터셋을 학습용과 검증용으로 분할
train_size = int(0.8 * len(train_dataset1))
val_size = len(train_dataset1) - train_size
train_dataset1, val_dataset1 = random_split(train_dataset1, [train_size, val_size])

# 분할된 데이터셋을 DataLoader에 로드할 수 있습니다.
train_loader1 = DataLoader(train_dataset1, batch_size=8, shuffle=True)
val_loader1 = DataLoader(val_dataset1, batch_size=8, shuffle=False)
test_loader1 = DataLoader(test_dataset1, batch_size=8, shuffle=False)

뒷과정은 동일 (새로운 모델 할당 후 동일 과정)